# Generate Map

> Class that generates the data map

In [ ]:
#| default_exp dash_components

In [ ]:
#| export
from nbdev.showdoc import *
# TODO work out how to get around below hack
try:
    from SolomonIslandsDataMap.load_data import *
except: 
    from load_data import *
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio # Unless this is used graphs will not be dynamic?
import json
from git import Repo
import pandas as pd
import numpy as np
from fastcore.test import *
from dash import Dash, dcc, Output, Input, html, Patch, ctx  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
from dash_bootstrap_templates import load_figure_template
import random
import dash_mantine_components as dmc

In [ ]:
# TODO - do a need a state class that tracks the state of different website objects to help with
# updating etc. 
# TODO - not even state, should it be an object that contains everythin to track. Execpt if I am patching,
# then the tracking in mem is unessary, so better to just track necesary values like: last changes card etc. 

## Load Data

In [ ]:

sol_geo = SolomonGeo.load_pickle("/testData/")
geo_df = sol_geo.geo_df

In [ ]:
#| export
# TODO I should build figures and maps in another script
def define_map(sol_df:SolomonGeo # Solomon geo object containing census data to input into map
                )->type(go.Figure()): # Returns a graph object figure
    '''
    Creates and returns the base cloreopath map
    '''
    # TODO - should I update this into a class with methods for updating
    # the other things? Acutally maybe as another function if the update is done through patch
    
    # cols_dd dictates the aggregation that will be visable
    cols_dd = sol_df.geo_levels
    # we need to add this to select which trace 
    # is going to be visible
    visible = np.array(cols_dd)
    # define traces and buttons at once
    traces = []
    buttons = []
    # TODO if fails remember I changed visible from cols_dd
    for value in cols_dd:
        traces.append(go.Choroplethmapbox(
                                geojson=sol_df.get_geojson(agg_filter = value),
                               locations=sol_df.get_df(agg_filter = value).index,
                               z = sol_df.get_df(agg_filter = value)['total_pop'],
                               colorscale="deep",
                                marker_line_width = 0.5,
                                zauto=True,
                visible= True if value==cols_dd[0] else False))
        
    # Show figure
    fig = go.Figure(data=traces)
    # This is in order to get the first title displayed correctly
    first_title = cols_dd[0]
    fig.update_layout(title=f"<b>{first_title}</b>",
                        title_x=0.5,
                        mapbox_style = 'carto-positron',
                        mapbox_zoom = 5,
                        mapbox_center={"lat": -9.565766, "lon": 162.012453},
                        margin={"r":0,"t":0,"l":0,"b":0},
                        clickmode = 'event+select',
    )
    
    return fig


In [ ]:
#| hide
show_doc(define_map)

---

[source](https://github.com/Gippers/SolomonIslandsDataMap/blob/main/SolomonIslandsDataMap/dash_components.py#L29){target="_blank" style="float:right; font-size:smaller"}

### define_map

>      define_map (sol_df:SolomonIslandsDataMap.load_data.SolomonGeo)

Creates and returns the base cloreopath map

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| sol_df | SolomonGeo | Solomon geo object containing census data to input into map |
| **Returns** | **Figure** | **Returns a graph object figure** |

In [ ]:
fig = define_map(sol_geo)

In [ ]:
fig['data'][0]['visible']

True

Create card group

In [ ]:
#| export
# todo - turn this eventually into a function

# TODO - make it in future so that clicking on a card updates the current census variable
# selection and it highlights it as clicked.
# TODO - workout how to make this into a collection of cards, potentially cardgroup
# TODO - need to rename this

def card_list(sg:SolomonGeo, # Input data object
                header:str, # Header of Accordian
                agg:str, #Desired aggregation of data in card
                var:str, # Desired variable to display in card
                loc:str = None, # Desired location within aggregation
                    )->[dbc.AccordionItem]: # Returns an accordian with selected data
    '''
    Create a list of cards to put in a cardgroup
    '''
    cards = []
    for var in sg.census_vars:
        cards.append(dbc.Col([
            dbc.Card(
            children = [
                dbc.CardHeader(
                    [html.H4(var)]

                ),
                dbc.CardBody(
                    [
                    html.H5(sg.get_df(agg_filter = agg, 
                                        var_filter = var, 
                                        loc_filter = loc).sum(), 
                            className = "text-center")
                    ] # TODO - in future will need to do weighted sum for some: might need an aggregate feature in class that 
                        # aggregates correctly based on the variable (i.e. sum or weighted sum), maybe work out in get_df function
                )]
            , class_name ="border-primary" #m-2 mb-3
            
            )])
        )

    return [dbc.AccordionItem(
                 dbc.Row(cards),
                title=header,
            )]

In [ ]:
#| hide
show_doc(card_list)

---

[source](https://github.com/Gippers/SolomonIslandsDataMap/blob/main/SolomonIslandsDataMap/dash_components.py#L78){target="_blank" style="float:right; font-size:smaller"}

### card_list

>      card_list (sg:SolomonIslandsDataMap.load_data.SolomonGeo, header:str,
>                 agg:str, var:str, loc:str=None)

Create a list of cards to put in a cardgroup

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| sg | SolomonGeo |  | Input data object |
| header | str |  | Header of Accordian |
| agg | str |  | Desired aggregation of data in card |
| var | str |  | Desired variable to display in card |
| loc | str | None | Desired location within aggregation |
| **Returns** | **[<class 'dash_bootstrap_components._components.AccordionItem.AccordionItem'>]** |  | **Returns an accordian with selected data** |

## Testing

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()